In [ ]:
!pip install transformers torch


In [ ]:
from transformers import pipeline
import pandas as pd


In [ ]:
# Load data
data = pd.read_csv('/content/Mistral_random_200_reviews_with_true_label_PD.csv')
# Display the first few rows
data.head()


,Cleaned_Review,Sentiment
0,i bought this phone as a replacement for the e...,Negative
1,bought phone june died already garbage,Negative
2,but within a couple of months they stopped wor...,Negative
3,replaced an aging motorolla g series with thus...,Neutral
4,the biggest problem is turning the yv on to of...,Negative


In [ ]:
# Initialize sentiment analysis pipeline with truncation
sentiment_analyzer = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment", truncation=True)
# Function

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
def analyze_sentiment(text):
    try:
        # Perform sentiment analysis
        result = sentiment_analyzer(text[:512])[0]  # Truncate to 512 characters
        label = result['label']
        # Map the model's output to positive, neutral, or negative labels
        if label == 'LABEL_2':  # Assuming LABEL_2 is positive
            return "Positive"
        elif label == 'LABEL_1':  # Assuming LABEL_1 is neutral
            return "Neutral"
        elif label == 'LABEL_0':  # Assuming LABEL_0 is negative
            return "Negative"
    except Exception as e:
        print(f"Error processing text: {text[:30]}... - {e}")
        return "Error"



In [ ]:
data['Sentiment'] = data['Cleaned_Review'].apply(analyze_sentiment)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
data.head()

,Cleaned_Review,Sentiment
0,i bought this phone as a replacement for the e...,Negative
1,bought phone june died already garbage,Negative
2,but within a couple of months they stopped wor...,Negative
3,replaced an aging motorolla g series with thus...,Neutral
4,the biggest problem is turning the yv on to of...,Negative


In [ ]:
!pip install -qU mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.7/229.7 kB 10.6 MB/s eta 0:00:00


In [ ]:
api = 'lnaHfIePcTFDeCbXyMdBMNTyYsyxxUxj'
#api ='aSwGSE2q6cWnXgFPyJ1XF1AOF3fsD4gB'

In [ ]:
import os
from mistralai import Mistral

api_key = api
model = "mistral-large-latest"

client = Mistral(api_key=api_key)



In [ ]:
data_1 = data.head(5)

In [ ]:

# Function to get sentiment from Mistral
def get_mistral_sentiment(review_text):
    chat_response = client.chat.complete(
        model=model,
        messages=[
            {
                "role": "user",
                "content": f"Simply answer whether the following text as in 1 word Positive, Negative, or Neutral: '{review_text}'",
            },
        ]
    )
    # Extract and return the response text
    return chat_response.choices[0].message.content.strip()

# Apply sentiment analysis to each review in the DataFrame
data_1['Mistral_Sentiment'] = data_1['Cleaned_Review'].apply(get_mistral_sentiment)

# Display the DataFrame with the new Mistral_Sentiment column
print(data_1.head())


SDKError: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}

In [ ]:
api_key = 'aSwGSE2q6cWnXgFPyJ1XF1AOF3fsD4gB'

In [ ]:
data_1.head()

,Cleaned_Review,Sentiment
0,i bought this phone as a replacement for the e...,Negative
1,bought phone june died already garbage,Negative
2,but within a couple of months they stopped wor...,Negative
3,replaced an aging motorolla g series with thus...,Neutral
4,the biggest problem is turning the yv on to of...,Negative


In [ ]:
# Function to get feature sentiment from Mistral
def get_mistral_feature_sentiment(review_text):
    chat_response = client.chat.complete(
        model=model,
        messages=[
            {
                "role": "user",
                "content": f"Identify key features from the review below and categorize each as Positive or Negative. Positive and Negative should be in Only Biagrams Format. Output should be in the format: Positive Features: [feature1, feature2, ...]; Negative Features: [feature1, feature2, ...]. Review text: '{review_text}'",
            },
        ]
    )
    # Extract and parse the response text
    response_text = chat_response.choices[0].message.content.strip()

    # Parse the response to separate positive and negative features
    positive_features = []
    negative_features = []

    # Split the response by labels
    if "Positive Features:" in response_text:
        positive_part = response_text.split("Positive Features:")[1].split("Negative Features:")[0].strip()
        positive_features = [feature.strip() for feature in positive_part.split(',') if feature]

    if "Negative Features:" in response_text:
        negative_part = response_text.split("Negative Features:")[1].strip()
        negative_features = [feature.strip() for feature in negative_part.split(',') if feature]

    return positive_features, negative_features

# Apply the function to each review and expand columns for positive and negative features
data_1[['Positive_Features', 'Negative_Features']] = data_1['Cleaned_Review'].apply(lambda x: pd.Series(get_mistral_feature_sentiment(x)))

# Display the DataFrame with new columns
print(data_1[['Cleaned_Review', 'Positive_Features', 'Negative_Features']].head())


SDKError: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}

In [ ]:
data_1.head()

In [ ]:
data_1.to_csv('/content/df.csv', index=False)